## I. Збір та анотування даних  
### Опрацюйте один із архівів Wiktionary і витягніть усі синонімні ряди для будь-якої мови. Будь ласка, не обирайте англійську, українську чи російську мови. 
Был выбран шведский язык.  
Пользовался последним дампом - `svwiktionary-latest-pages-articles.xml`  
Скрипт - `parsing.py`  
Результат выполнения скрипта:

In [ ]:
import pandas as pd

In [9]:
unpickled_df = pd.read_pickle("./syns.pkl")
unpickled_df

,word,syn
0,besserwisser['know-it-all'],"[bättrevetare, viktigpetter, messerschmitt]"
1,Sverige['Sweden'],"[landet lagom, moder Svea, Svea rike, Konungar..."
2,barr['barre'],"[balettstång, stång]"
3,samlag['intercourse'],"[sex, knull, älskog]"
4,"knulla['fuck', 'screw', 'have sex']","[banga, borra, bulta, doppa, dra över, dunka, ..."
...,...,...
5493,"pilates boll['exercise ball', 'yoga ball', 'gy...",[balansboll]
5494,växt ätande['herbivorous'],[herbivor]
5495,storhetsvansinne|storhetsvansinn -ig|ig['megal...,"[megaloman, högmodig, övermodig]"
5496,måne|mån kalv['mooncalf'],[druvbörd]


### Визначте рівень згоди між анотувальниками (або inter-annotator agreement) у корпусі NUCLE Error Corpus. 

In [3]:
from itertools import combinations

tags = "Any Vt3 Vm V0 Vform SVA ArtOrDet Nn Npos Pform Pref Prep Wci Wa Wform Wtone Srun Smod Spar Sfrag Ssub WOinc WOadv Trans Mec Rloc Cit Others Um".split(" ")

lines = []
with open("official-2014.combined-withalt.m2") as file:
    lines = file.readlines()

comb = list(combinations(range(5), 2))

data_for_tag = {}
for t in tags:
    data_for_tag[t]={}

#prepare data 
for t in tags:
    for c in comb:
        data_for_tag[t][c]=[]

examples = []
curr = []
for line in lines:
    if line!="\n":
        curr.append(line)
    else:
        examples.append(curr)
        curr = [] 

def create_dict():
    d = {}
    for tag in tags: 
        d[tag] = []
    return d

def get_anotator(s):
    return int(s.strip()[-1])

def get_anotation_string(s, sentense_number, tag):
    i = s.find("REQUIRED")
    s = s.strip()[2:i]
    token_range = ""
    count = 0
    for x in s:
        if x=="|":
            break
        count += 1
        token_range += x
    s = s[count+3:]
    count = 0 
    anotation_tag=""
    for x in s:
        if x=="|":
            break
        anotation_tag += x
        count += 1
    s = s[count+3:]
    if tag!="Any":
        if anotation_tag!=tag:
            return ""
    correction=""
    count=0
    for x in s:
        if x=="|":
            break
        correction+=x
        count+=1
    return " ".join([str(sentense_number), token_range, correction])


def process_block(l, index):
    anotations = l[1:]
    if len(anotations) > 1:
        anotation_strings = create_dict()
        t = [""]*5
        curr_anotator = -1
        for x in anotations:
            an = get_anotator(x)
            if an != curr_anotator:
                if curr_anotator!=-1:
                    t[curr_anotator] = anotation_strings
                    anotation_strings = create_dict()
                curr_anotator = an
            for tag in tags:
                anotation_strings[tag].append(get_anotation_string(x, index, tag))

        t[curr_anotator] = anotation_strings
        #append processed lines to data 
        indexes = [i for i, j in enumerate(t) if j!='']
        if len(indexes)>1:
            for tag in tags:
                for idx in combinations(indexes, 2):
                    d1 = t[idx[0]][tag]
                    d2 = t[idx[1]][tag]
                    g = [d2.count(el) for el in d1 if el!='']
                    if len(g)!=0:
                        c = sum(g)*2/(len(d1)+len(d2))
                        data_for_tag[tag][idx].append(c)


for i, e in enumerate(examples):
    process_block(e,i)


#calculating result 
for tag in tags:
    data = data_for_tag[tag]
    combined_score = 0
    count = 0
    for k in data:
        d = data[k]
        if len(d)!=0: # Means this annotators annotated same sentences with this tag
            combined_score += sum(d)/len(d)
            count += 1

    res = combined_score/count if count!=0 else 0
    print("For tag: {} score is: {}".format(tag, res))

For tag: Any score is: 0.4339096067012667
For tag: Vt3 score is: 0
For tag: Vm score is: 0.059583316422409
For tag: V0 score is: 0.08932971231964114
For tag: Vform score is: 0.1371039023483001
For tag: SVA score is: 0.10700282848347095
For tag: ArtOrDet score is: 0.1578472245092778
For tag: Nn score is: 0.16807668845741963
For tag: Npos score is: 0.05250616612549034
For tag: Pform score is: 0.0683225705034294
For tag: Pref score is: 0.08492123276948227
For tag: Prep score is: 0.15783307874541927
For tag: Wci score is: 0.14062958379625046
For tag: Wa score is: 0.07407407407407407
For tag: Wform score is: 0.10619317510807695
For tag: Wtone score is: 0.08240740740740742
For tag: Srun score is: 0.09647022829618404
For tag: Smod score is: 0.06666666666666667
For tag: Spar score is: 0.012630975245411865
For tag: Sfrag score is: 0.013494318181818182
For tag: Ssub score is: 0.08436944540097073
For tag: WOinc score is: 0.034409080881520476
For tag: WOadv score is: 0.04827790265290266
For tag: T

Conclusions:
Multiple,  different  corrections  are  of-ten acceptable. In general considering all tags score showed moderate agreement between peers. 